<a href="https://colab.research.google.com/github/vanikanjirangat/NLP_DI_NADI/blob/main/NADI_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==3.0.0

In [ ]:
import tensorflow as tf
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import BertTokenizer
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
import matplotlib.pyplot as plt
import ast
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
le = LabelEncoder()
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import accuracy_score, f1_score




In [ ]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
torch.cuda.get_device_name(0)

Found GPU at: /device:GPU:0


'Tesla T4'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [ ]:
root_dir = "/content/gdrive/My Drive/"
input_dir=os.path.join(root_dir,'NADI')

**AraBERT model** : For running the AraBERT model use below code

In [ ]:
import time
import datetime
from transformers import AutoModel, AutoTokenizer, AutoConfig,AutoModelForSequenceClassification
class Model:
    def __init__(self,path):
        # self.args = args
        self.path=path
        self.MAX_LEN=128
        # self.tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.tokenizer = AutoTokenizer.from_pretrained('aubmindlab/bert-base-arabert')
        #bashar-talafha/multi-dialect-bert-base-arabic
        num_labels=18
        self.config = AutoConfig.from_pretrained('aubmindlab/bert-base-arabert',num_labels=num_labels)
        # if not os.path.isdir(self.opath):
        #     os.makedirs(self.opath)
 

    # def format_time(elapsed):
    #   elapsed_rounded = int(round((elapsed)))
    #   return str(datetime.timedelta(seconds=elapsed_rounded))
    
            
    def extract_data(self,name,XY=None):
        file =self.path+name
        df = pd.read_csv(file, delimiter='\t')
        df.replace(np.nan,'NIL', inplace=True)
        sentences= df["#2_content"].values
        labels = df["#3_label"].values
        return (sentences,labels)
    def extract_data_test(self,name,XY=None):
      file =self.path+name
      df = pd.read_csv(file, delimiter='\t')
      df.replace(np.nan,'NIL', inplace=True)
      sentences= df["#2_content"].values
      #labels = df["#3_label"].values
      return sentences
    

    def process_inputs(self,sentences,labels):
      sentences= [self.tokenizer.encode_plus(sent,add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]
      # sentence_idx = np.linspace(0,len(sentences), len(sentences),False)
      # torch_idx = torch.tensor(sentence_idx)
      tags_vals = list(labels)
      le.fit(labels)
      le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
      labels=le.fit_transform(labels)
      
      print(le_name_mapping)
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [inputs["input_ids"] for inputs in sentences]

      # Pad our input tokens
      input_ids = pad_sequences(input_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")
      attention_masks = []

      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask= [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

        
      # token_type_ids=[inputs["token_type_ids"] for inputs in sentences]
      # token_type_ids=pad_sequences(token_type_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")

      inputs, labels = input_ids, labels
      masks,_= attention_masks, input_ids
      # Convert all of our data into torch tensors, the required datatype for our model

      self.inputs = torch.tensor(inputs).to(torch.int64)
      # validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
      self.labels = torch.tensor(labels).to(torch.int64)
      # validation_labels = torch.tensor(validation_labels).to(torch.int64)
      self.masks = torch.tensor(masks).to(torch.int64)
      # validation_masks = torch.tensor(validation_masks).to(torch.int64)
      # self.types=torch.tensor(types).to(torch.int64)
      self.data = TensorDataset(self.inputs,self.masks, self.labels)
      self.sampler = RandomSampler(self.data)
      self.dataloader = DataLoader(self.data, sampler=self.sampler, batch_size=32)

      # return (self.inputs,self.labels,self.masks,self.types)
    def process_dev_inputs(self,sentences,labels):
      sentences= [self.tokenizer.encode_plus(sent,add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]
      # sentence_idx = np.linspace(0,len(sentences), len(sentences),False)
      # torch_idx = torch.tensor(sentence_idx)
      tags_vals = list(labels)
      le.fit(labels)
      le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
      labels=le.fit_transform(labels)
      
      print(le_name_mapping)
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [inputs["input_ids"] for inputs in sentences]

      # Pad our input tokens
      input_ids = pad_sequences(input_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")
      attention_masks = []

      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask= [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

        
      # token_type_ids=[inputs["token_type_ids"] for inputs in sentences]
      # token_type_ids=pad_sequences(token_type_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")

      inputs, labels = input_ids, labels
      masks,_= attention_masks, input_ids
      # Convert all of our data into torch tensors, the required datatype for our model

      self.inputs = torch.tensor(inputs).to(torch.int64)
      # validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
      self.labels = torch.tensor(labels).to(torch.int64)
      # validation_labels = torch.tensor(validation_labels).to(torch.int64)
      self.masks = torch.tensor(masks).to(torch.int64)
      # validation_masks = torch.tensor(validation_masks).to(torch.int64)
      # self.types=torch.tensor(types).to(torch.int64)
      self.data = TensorDataset(self.inputs,self.masks, self.labels)
      self.sampler = RandomSampler(self.data)
      self.validationdataloader = DataLoader(self.data, sampler=self.sampler, batch_size=32)

    # def process_inputs_test(self,sentences,labels,act_ids,batch_size=1):
    #   sentences= [self.tokenizer.encode_plus(sent,add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]
    #   sentence_idx = np.linspace(0,len(sentences), len(sentences),False)
    #   torch_idx = torch.tensor(sentence_idx)
    #   tags_vals = list(labels)
    #   le.fit(labels)
    #   le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    #   labels=le.fit_transform(labels)
      
    #   print(le_name_mapping)
    #   # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    #   input_ids = [inputs["input_ids"] for inputs in sentences]

    #   # Pad our input tokens
    #   input_ids = pad_sequences(input_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")
    #   attention_masks = []

    #   # Create a mask of 1s for each token followed by 0s for padding
    #   for seq in input_ids:
    #     seq_mask= [float(i>0) for i in seq]
    #     attention_masks.append(seq_mask)

        
    #   # token_type_ids=[inputs["token_type_ids"] for inputs in sentences]
    #   # token_type_ids=pad_sequences(token_type_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")

    #   inputs, labels = input_ids, labels
    #   masks,_= attention_masks, input_ids
    #   # Convert all of our data into torch tensors, the required datatype for our model

    #   self.inputs = torch.tensor(inputs).to(torch.int64)
    #   # validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
    #   self.labels = torch.tensor(labels).to(torch.int64)
    #   # validation_labels = torch.tensor(validation_labels).to(torch.int64)
    #   self.act_ids = torch.tensor(act_ids).to(torch.int64)
    #   # validation_labels = torch.tensor(validation_labels).to(torch.int64)
    #   self.masks = torch.tensor(masks).to(torch.int64)
    #   self.torch_idx = torch.tensor(sentence_idx).to(torch.int64)
    #   self.data = TensorDataset(self.inputs,self.masks, self.labels,self.torch_idx,self.act_ids)
    #   self.sampler = RandomSampler(self.data)
    #   self.dataloader = DataLoader(self.data, sampler=self.sampler, batch_size=batch_size)
    def process_inputs_test(self,sentences,batch_size=1):
      sentences= [self.tokenizer.encode_plus(sent,add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]
      sentence_idx = np.linspace(0,len(sentences), len(sentences),False)
      torch_idx = torch.tensor(sentence_idx)
      # tags_vals = list(labels)
      # le.fit(labels)
      # le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
      # labels=le.fit_transform(labels)
      
      # print(le_name_mapping)
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [inputs["input_ids"] for inputs in sentences]

      # Pad our input tokens
      input_ids = pad_sequences(input_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")
      attention_masks = []

      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask= [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

        
      # token_type_ids=[inputs["token_type_ids"] for inputs in sentences]
      # token_type_ids=pad_sequences(token_type_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")

      inputs = input_ids
      masks,_= attention_masks, input_ids
      # Convert all of our data into torch tensors, the required datatype for our model

      self.inputs = torch.tensor(inputs).to(torch.int64)
      # validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
      #self.labels = torch.tensor(labels).to(torch.int64)
      # validation_labels = torch.tensor(validation_labels).to(torch.int64)
      #self.act_ids = torch.tensor(act_ids).to(torch.int64)
      # validation_labels = torch.tensor(validation_labels).to(torch.int64)
      self.masks = torch.tensor(masks).to(torch.int64)
      self.torch_idx = torch.tensor(sentence_idx).to(torch.int64)
      self.data = TensorDataset(self.inputs,self.masks)
      self.sampler = RandomSampler(self.data)
      self.dataloader = DataLoader(self.data, sampler=self.sampler, batch_size=batch_size)

    def train_save_load(self,train=1,retrain=0,label_smoothing = -1,XY=None):
      
      #WEIGHTS_NAME = "GDI2018_calibrateTrain4.bin"
      WEIGHTS_NAME = "NADI_AraBERTTestA.bin"
      # WEIGHTS_NAME = "GDI2018_mbertTrain4.bin"
      # WEIGHTS_NAME='GDITrain4_Bert_base_cased1.bin'
      # WEIGHTS_NAME = "GDI2018_calibrateTrain4LS_01.bin"#calibrated with LS 0.1
      # WEIGHTS_NAME = "GDI2018_calibrateTrain4LS.bin"
      # WEIGHTS_NAME = "GDI_VBERT_cased.bin"

      OUTPUT_DIR = input_dir
      output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
      if retrain!=1:

        
       
        # self.model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=5)
        self.model = AutoModelForSequenceClassification.from_pretrained('aubmindlab/bert-base-arabert',config=self.config)
      else:
        # self.model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=4)
        self.model = AutoModelForSequenceClassification.from_pretrained('aubmindlab/bert-base-arabert',config=self.config)
        state_dict = torch.load(output_model_file)
        self.model.load_state_dict(state_dict)
        WEIGHTS_NAME = "NADI_AraBERT_inc.bin"

        OUTPUT_DIR = input_dir
        output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
      self.model.cuda()
      param_optimizer = list(self.model.named_parameters())
      no_decay = ['bias', 'gamma', 'beta']
      optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},{'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                                                                                                                                                     'weight_decay_rate': 0.0}]
      #optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5)
      optimizer = AdamW(optimizer_grouped_parameters,lr=5e-5)
      #optimizer = AdamW(optimizer_grouped_parameters,lr=3e-5)
      
      
      train_loss_set = []
      out_train={}
      batch_size=32
      true=[]
      logits_all=[]
      output_dicts = []
      batch_size=32
      epochs = 4
      import time
      start_time = time.time()
      if train==1:
        for _ in trange(epochs, desc="Epoch"):
          # Trainin
          # Set our model to training mode (as opposed to evaluation mode
          self.model.train()
          # Tracking variables
          tr_loss = 0
          nb_tr_examples, nb_tr_steps = 0, 0
          # Train the data for one epoch
          for step, batch in enumerate(self.dataloader):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            b_input_ids,b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            # loss = model(b_input_ids, token_type_ids=b_types, attention_mask=b_input_mask, labels=b_labels)
            loss,logits= self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            if label_smoothing == -1:
              logits=logits
            else:
              criterion = LabelSmoothingLoss(label_smoothing)
              loss=criterion(logits,b_labels)
            


       
            train_loss_set.append(loss.item())    
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1
          print("Train loss: {}".format(tr_loss/nb_tr_steps))
        print("--- %s seconds ---" % (time.time() - start_time)) 
        torch.save(self.model.state_dict(), output_model_file)
        
      else:
        state_dict = torch.load(output_model_file)
        self.model.load_state_dict(state_dict) 
      return output_dicts

    def eval(self,label_smoothing = -1):
      batch_size=32
      eval_loss = 0
      # Put model in evaluation mod
      self.model.eval()
      # Tracking variables 
      self.predictions , self.true_labels = [], []
      output_dicts=[]
     
      
      for batch in self.dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids,b_input_mask, b_labels = batch
        # # Telling the model not to compute or store gradients, saving memory and speeding up prediction
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          if label_smoothing == -1:
            logits=outputs[0]
            loss=outputs[1]
          else:
            criterion = LabelSmoothingLoss(label_smoothing)
            loss=criterion(logits,b_labels)
          
          eval_loss += loss.item()
          self.dataloader.set_description(f'eval loss = {(eval_loss / i):.6f}')
      return eval_loss / len(self.dataloader)
    
    def simple_test(self):
      batch_size=32
      # Put model in evaluation mod
      self.model.eval()
      # Tracking variables 
      self.predictions , self.true_labels = [], []
      output_dicts=[]
      
      
      for batch in self.dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids,b_input_mask= batch
        # # Telling the model not to compute or store gradients, saving memory and speeding up prediction
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          logits=outputs[0]
          for j in range(logits.size(0)):
            probs = F.softmax(logits[j], -1)
            output_dict = {
                # 'index': batch_size * i + j,
                #'true': b_labels[j].cpu().numpy().tolist(),
                'pred': logits[j].argmax().item(),
                'conf': probs.max().item(),
                'logits': logits[j].cpu().numpy().tolist(),
                'probs': probs.cpu().numpy().tolist(),
            }
            output_dicts.append(output_dict)
      #y_true = [output_dict['true'] for output_dict in output_dicts]
      y_pred = [output_dict['pred'] for output_dict in output_dicts]
      #y_conf = [output_dict['conf'] for output_dict in output_dicts]

      #accuracy = accuracy_score(y_true, y_pred) * 100.
      #f1 = f1_score(y_true, y_pred, average='macro') * 100.
      #confidence = np.mean(y_conf) * 100.

      # results_dict = {
      #     'accuracy': accuracy_score(y_true, y_pred) * 100.,
      #     'macro-F1': f1_score(y_true, y_pred, average='macro') * 100.,
      #     'confidence': np.mean(y_conf) * 100.,
      # }
      # print(results_dict)
      # print(classification_report(y_true,y_pred))
      # print(confusion_matrix(y_true,y_pred))
      print("writing the results...")
      class_map={'algeria': 0, 'bahrain': 1, 'egypt': 2, 'iraq': 3, 'jordan': 4, 'ksa': 5, 'kuwait': 6, 'lebanon': 7, 'libya': 8, 'morocco': 9, 'oman': 10, 'palestine': 11, 'qatar': 12, 'sudan': 13, 'syria': 14, 'tunisia': 15, 'uae': 16, 'yemen': 17}
      c={}
      for i in class_map:
        c[class_map[i]]=i
      p=[]
      #a=[]
      for i,k in enumerate(y_pred):
          p.append(c[k])
          #a.append(c[y_true[i]])
      path=input_dir+"/official_results/testa_1.txt"
      #gold_path=input_dir+"/results/AraBERTgold.txt"
      with open(path,"w") as f:
        for r in p:
          f.write(str(r))
          f.write("\n")
      # with open(gold_path,"w") as f:
      #   for r in a:
      #     f.write(str(r))
      #     f.write("\n")
      return output_dicts

    
      

**Multi-dialect-Arabic-BERT**:For running the Multi-dialect-Arabic-BERT model use below code:

In [ ]:
import time
import datetime
from transformers import AutoModel, AutoTokenizer, AutoConfig,AutoModelForSequenceClassification
class Model:
    def __init__(self,path):
        # self.args = args
        self.path=path
        self.MAX_LEN=128
        # self.tokenizer=BertTokenizer.from_pretrained('bert-base-multilingual-cased')
        self.tokenizer = AutoTokenizer.from_pretrained('bashar-talafha/multi-dialect-bert-base-arabic')
        #bashar-talafha/multi-dialect-bert-base-arabic
        num_labels=18
        self.config = AutoConfig.from_pretrained('bashar-talafha/multi-dialect-bert-base-arabic',num_labels=num_labels)
        # if not os.path.isdir(self.opath):
        #     os.makedirs(self.opath)
 

    # def format_time(elapsed):
    #   elapsed_rounded = int(round((elapsed)))
    #   return str(datetime.timedelta(seconds=elapsed_rounded))
    
            
    def extract_data(self,name,XY=None):
        file =self.path+name
        df = pd.read_csv(file, delimiter='\t')
        df.replace(np.nan,'NIL', inplace=True)
        sentences= df["#2_content"].values
        labels = df["#3_label"].values
        return (sentences,labels)
    def extract_data_test(self,name,XY=None):
      file =self.path+name
      df = pd.read_csv(file, delimiter='\t')
      df.replace(np.nan,'NIL', inplace=True)
      sentences= df["#2_content"].values
      #labels = df["#3_label"].values
      return sentences
    

    def process_inputs(self,sentences,labels):
      sentences= [self.tokenizer.encode_plus(sent,add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]
      # sentence_idx = np.linspace(0,len(sentences), len(sentences),False)
      # torch_idx = torch.tensor(sentence_idx)
      tags_vals = list(labels)
      le.fit(labels)
      le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
      labels=le.fit_transform(labels)
      
      print(le_name_mapping)
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [inputs["input_ids"] for inputs in sentences]

      # Pad our input tokens
      input_ids = pad_sequences(input_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")
      attention_masks = []

      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask= [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

        
      # token_type_ids=[inputs["token_type_ids"] for inputs in sentences]
      # token_type_ids=pad_sequences(token_type_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")

      inputs, labels = input_ids, labels
      masks,_= attention_masks, input_ids
      # Convert all of our data into torch tensors, the required datatype for our model

      self.inputs = torch.tensor(inputs).to(torch.int64)
      # validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
      self.labels = torch.tensor(labels).to(torch.int64)
      # validation_labels = torch.tensor(validation_labels).to(torch.int64)
      self.masks = torch.tensor(masks).to(torch.int64)
      # validation_masks = torch.tensor(validation_masks).to(torch.int64)
      # self.types=torch.tensor(types).to(torch.int64)
      self.data = TensorDataset(self.inputs,self.masks, self.labels)
      self.sampler = RandomSampler(self.data)
      self.dataloader = DataLoader(self.data, sampler=self.sampler, batch_size=32)

      # return (self.inputs,self.labels,self.masks,self.types)
    def process_dev_inputs(self,sentences,labels):
      sentences= [self.tokenizer.encode_plus(sent,add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]
      # sentence_idx = np.linspace(0,len(sentences), len(sentences),False)
      # torch_idx = torch.tensor(sentence_idx)
      tags_vals = list(labels)
      le.fit(labels)
      le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
      labels=le.fit_transform(labels)
      
      print(le_name_mapping)
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [inputs["input_ids"] for inputs in sentences]

      # Pad our input tokens
      input_ids = pad_sequences(input_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")
      attention_masks = []

      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask= [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

        
      # token_type_ids=[inputs["token_type_ids"] for inputs in sentences]
      # token_type_ids=pad_sequences(token_type_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")

      inputs, labels = input_ids, labels
      masks,_= attention_masks, input_ids
      # Convert all of our data into torch tensors, the required datatype for our model

      self.inputs = torch.tensor(inputs).to(torch.int64)
      # validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
      self.labels = torch.tensor(labels).to(torch.int64)
      # validation_labels = torch.tensor(validation_labels).to(torch.int64)
      self.masks = torch.tensor(masks).to(torch.int64)
      # validation_masks = torch.tensor(validation_masks).to(torch.int64)
      # self.types=torch.tensor(types).to(torch.int64)
      self.data = TensorDataset(self.inputs,self.masks, self.labels)
      self.sampler = RandomSampler(self.data)
      self.validationdataloader = DataLoader(self.data, sampler=self.sampler, batch_size=32)

    # def process_inputs_test(self,sentences,labels,act_ids,batch_size=1):
    #   sentences= [self.tokenizer.encode_plus(sent,add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]
    #   sentence_idx = np.linspace(0,len(sentences), len(sentences),False)
    #   torch_idx = torch.tensor(sentence_idx)
    #   tags_vals = list(labels)
    #   le.fit(labels)
    #   le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
    #   labels=le.fit_transform(labels)
      
    #   print(le_name_mapping)
    #   # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
    #   input_ids = [inputs["input_ids"] for inputs in sentences]

    #   # Pad our input tokens
    #   input_ids = pad_sequences(input_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")
    #   attention_masks = []

    #   # Create a mask of 1s for each token followed by 0s for padding
    #   for seq in input_ids:
    #     seq_mask= [float(i>0) for i in seq]
    #     attention_masks.append(seq_mask)

        
    #   # token_type_ids=[inputs["token_type_ids"] for inputs in sentences]
    #   # token_type_ids=pad_sequences(token_type_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")

    #   inputs, labels = input_ids, labels
    #   masks,_= attention_masks, input_ids
    #   # Convert all of our data into torch tensors, the required datatype for our model

    #   self.inputs = torch.tensor(inputs).to(torch.int64)
    #   # validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
    #   self.labels = torch.tensor(labels).to(torch.int64)
    #   # validation_labels = torch.tensor(validation_labels).to(torch.int64)
    #   self.act_ids = torch.tensor(act_ids).to(torch.int64)
    #   # validation_labels = torch.tensor(validation_labels).to(torch.int64)
    #   self.masks = torch.tensor(masks).to(torch.int64)
    #   self.torch_idx = torch.tensor(sentence_idx).to(torch.int64)
    #   self.data = TensorDataset(self.inputs,self.masks, self.labels,self.torch_idx,self.act_ids)
    #   self.sampler = RandomSampler(self.data)
    #   self.dataloader = DataLoader(self.data, sampler=self.sampler, batch_size=batch_size)
    def process_inputs_test(self,sentences,batch_size=1):
      sentences= [self.tokenizer.encode_plus(sent,add_special_tokens=True, max_length=self.MAX_LEN,truncation='longest_first') for i,sent in enumerate(sentences)]
      sentence_idx = np.linspace(0,len(sentences), len(sentences),False)
      torch_idx = torch.tensor(sentence_idx)
      # tags_vals = list(labels)
      # le.fit(labels)
      # le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
      # labels=le.fit_transform(labels)
      
      # print(le_name_mapping)
      # Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
      input_ids = [inputs["input_ids"] for inputs in sentences]

      # Pad our input tokens
      input_ids = pad_sequences(input_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")
      attention_masks = []

      # Create a mask of 1s for each token followed by 0s for padding
      for seq in input_ids:
        seq_mask= [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

        
      # token_type_ids=[inputs["token_type_ids"] for inputs in sentences]
      # token_type_ids=pad_sequences(token_type_ids, maxlen=self.MAX_LEN,truncating="post", padding="post")

      inputs = input_ids
      masks,_= attention_masks, input_ids
      # Convert all of our data into torch tensors, the required datatype for our model

      self.inputs = torch.tensor(inputs).to(torch.int64)
      # validation_inputs = torch.tensor(validation_inputs).to(torch.int64)
      #self.labels = torch.tensor(labels).to(torch.int64)
      # validation_labels = torch.tensor(validation_labels).to(torch.int64)
      #self.act_ids = torch.tensor(act_ids).to(torch.int64)
      # validation_labels = torch.tensor(validation_labels).to(torch.int64)
      self.masks = torch.tensor(masks).to(torch.int64)
      self.torch_idx = torch.tensor(sentence_idx).to(torch.int64)
      self.data = TensorDataset(self.inputs,self.masks)
      self.sampler = RandomSampler(self.data)
      self.dataloader = DataLoader(self.data, sampler=self.sampler, batch_size=batch_size)

    def train_save_load(self,train=1,retrain=0,label_smoothing = -1,XY=None):
      
      #WEIGHTS_NAME = "GDI2018_calibrateTrain4.bin"
      WEIGHTS_NAME = "NADI_multidialectAraBERT.bin"
      # WEIGHTS_NAME = "GDI2018_mbertTrain4.bin"
      # WEIGHTS_NAME='GDITrain4_Bert_base_cased1.bin'
      # WEIGHTS_NAME = "GDI2018_calibrateTrain4LS_01.bin"#calibrated with LS 0.1
      # WEIGHTS_NAME = "GDI2018_calibrateTrain4LS.bin"
      # WEIGHTS_NAME = "GDI_VBERT_cased.bin"

      OUTPUT_DIR = input_dir
      output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
      if retrain!=1:

        
       
        # self.model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=5)
        self.model = AutoModelForSequenceClassification.from_pretrained('bashar-talafha/multi-dialect-bert-base-arabic',config=self.config)
      else:
        # self.model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=4)
        self.model = AutoModelForSequenceClassification.from_pretrained('bashar-talafha/multi-dialect-bert-base-arabic',config=self.config)
        state_dict = torch.load(output_model_file)
        self.model.load_state_dict(state_dict)
        WEIGHTS_NAME = "NADI_AraBERTdialect_inc.bin"

        OUTPUT_DIR = input_dir
        output_model_file = os.path.join(OUTPUT_DIR, WEIGHTS_NAME)
      self.model.cuda()
      param_optimizer = list(self.model.named_parameters())
      no_decay = ['bias', 'gamma', 'beta']
      optimizer_grouped_parameters = [{'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)], 'weight_decay_rate': 0.01},{'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
                                                                                                                                                     'weight_decay_rate': 0.0}]
      #optimizer = AdamW(optimizer_grouped_parameters,lr=2e-5)
      optimizer = AdamW(optimizer_grouped_parameters,lr=5e-5)
      #optimizer = AdamW(optimizer_grouped_parameters,lr=3e-5)
      
      
      train_loss_set = []
      out_train={}
      batch_size=32
      true=[]
      logits_all=[]
      output_dicts = []
      batch_size=32
      epochs = 4
      import time
      start_time = time.time()
      if train==1:
        for _ in trange(epochs, desc="Epoch"):
          # Trainin
          # Set our model to training mode (as opposed to evaluation mode
          self.model.train()
          # Tracking variables
          tr_loss = 0
          nb_tr_examples, nb_tr_steps = 0, 0
          # Train the data for one epoch
          for step, batch in enumerate(self.dataloader):
            # Add batch to GPU
            batch = tuple(t.to(device) for t in batch)
            b_input_ids,b_input_mask, b_labels = batch
            # Clear out the gradients (by default they accumulate)
            optimizer.zero_grad()
            # Forward pass
            # loss = model(b_input_ids, token_type_ids=b_types, attention_mask=b_input_mask, labels=b_labels)
            loss,logits= self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
            if label_smoothing == -1:
              logits=logits
            else:
              criterion = LabelSmoothingLoss(label_smoothing)
              loss=criterion(logits,b_labels)
            


       
            train_loss_set.append(loss.item())    
            # Backward pass
            loss.backward()
            # Update parameters and take a step using the computed gradient
            optimizer.step()
            # Update tracking variables
            tr_loss += loss.item()
            nb_tr_examples += b_input_ids.size(0)
            nb_tr_steps += 1
          print("Train loss: {}".format(tr_loss/nb_tr_steps))
        print("--- %s seconds ---" % (time.time() - start_time)) 
        torch.save(self.model.state_dict(), output_model_file)
        
      else:
        state_dict = torch.load(output_model_file)
        self.model.load_state_dict(state_dict) 
      return output_dicts

    def eval(self,label_smoothing = -1):
      batch_size=32
      eval_loss = 0
      # Put model in evaluation mod
      self.model.eval()
      # Tracking variables 
      self.predictions , self.true_labels = [], []
      output_dicts=[]
     
      
      for batch in self.dataloader:
        
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids,b_input_mask, b_labels = batch
        # # Telling the model not to compute or store gradients, saving memory and speeding up prediction
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          if label_smoothing == -1:
            logits=outputs[0]
            loss=outputs[1]
          else:
            criterion = LabelSmoothingLoss(label_smoothing)
            loss=criterion(logits,b_labels)
          
          eval_loss += loss.item()
          self.dataloader.set_description(f'eval loss = {(eval_loss / i):.6f}')
      return eval_loss / len(self.dataloader)
    
    # def simple_test(self):
    #   batch_size=32
    #   # Put model in evaluation mod
    #   self.model.eval()
    #   # Tracking variables 
    #   self.predictions , self.true_labels = [], []
    #   output_dicts=[]
      
      
    #   for batch in self.dataloader:
    #     # Add batch to GPU
    #     batch = tuple(t.to(device) for t in batch)
    #     # Unpack the inputs from our dataloader
    #     b_input_ids,b_input_mask,b_labels= batch
    #     # # Telling the model not to compute or store gradients, saving memory and speeding up prediction
    #     with torch.no_grad():
    #       # Forward pass, calculate logit predictions
    #       outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    #       logits=outputs[0]
    #       for j in range(logits.size(0)):
    #         probs = F.softmax(logits[j], -1)
    #         output_dict = {
    #             # 'index': batch_size * i + j,
    #             'true': b_labels[j].cpu().numpy().tolist(),
    #             'pred': logits[j].argmax().item(),
    #             'conf': probs.max().item(),
    #             'logits': logits[j].cpu().numpy().tolist(),
    #             'probs': probs.cpu().numpy().tolist(),
    #         }
    #         output_dicts.append(output_dict)
    #   y_true = [output_dict['true'] for output_dict in output_dicts]
    #   y_pred = [output_dict['pred'] for output_dict in output_dicts]
    #   y_conf = [output_dict['conf'] for output_dict in output_dicts]

    #   accuracy = accuracy_score(y_true, y_pred) * 100.
    #   f1 = f1_score(y_true, y_pred, average='macro') * 100.
    #   confidence = np.mean(y_conf) * 100.

    #   results_dict = {
    #       'accuracy': accuracy_score(y_true, y_pred) * 100.,
    #       'macro-F1': f1_score(y_true, y_pred, average='macro') * 100.,
    #       'confidence': np.mean(y_conf) * 100.,
    #   }
    #   print(results_dict)
    #   print(classification_report(y_true,y_pred))
    #   print(confusion_matrix(y_true,y_pred))
    #   print("writing the results...")
    #   class_map={'algeria': 0, 'bahrain': 1, 'egypt': 2, 'iraq': 3, 'jordan': 4, 'ksa': 5, 'kuwait': 6, 'lebanon': 7, 'libya': 8, 'morocco': 9, 'oman': 10, 'palestine': 11, 'qatar': 12, 'sudan': 13, 'syria': 14, 'tunisia': 15, 'uae': 16, 'yemen': 17}
    #   c={}
    #   for i in class_map:
    #     c[class_map[i]]=i
    #   p=[]
    #   #a=[]
    #   for i,k in enumerate(y_pred):
    #       p.append(c[k])
    #       #a.append(c[y_true[i]])
    #   path=input_dir+"/results/NADI_dialect_BERT.txt"
    #   #gold_path=input_dir+"/results/AraBERTgold.txt"
    #   with open(path,"w") as f:
    #     for r in p:
    #       f.write(str(r))
    #       f.write("\n")
    #   # with open(gold_path,"w") as f:
    #   #   for r in a:
    #   #     f.write(str(r))
    #   #     f.write("\n")
    #   return output_dicts

    def simple_test(self):
      batch_size=32
      # Put model in evaluation mod
      self.model.eval()
      # Tracking variables 
      self.predictions , self.true_labels = [], []
      output_dicts=[]
      
      
      for batch in self.dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)
        # Unpack the inputs from our dataloader
        b_input_ids,b_input_mask= batch
        # # Telling the model not to compute or store gradients, saving memory and speeding up prediction
        with torch.no_grad():
          # Forward pass, calculate logit predictions
          outputs = self.model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
          logits=outputs[0]
          for j in range(logits.size(0)):
            probs = F.softmax(logits[j], -1)
            output_dict = {
                # 'index': batch_size * i + j,
                #'true': b_labels[j].cpu().numpy().tolist(),
                'pred': logits[j].argmax().item(),
                'conf': probs.max().item(),
                'logits': logits[j].cpu().numpy().tolist(),
                'probs': probs.cpu().numpy().tolist(),
            }
            output_dicts.append(output_dict)
      #y_true = [output_dict['true'] for output_dict in output_dicts]
      y_pred = [output_dict['pred'] for output_dict in output_dicts]
      #y_conf = [output_dict['conf'] for output_dict in output_dicts]

      #accuracy = accuracy_score(y_true, y_pred) * 100.
      #f1 = f1_score(y_true, y_pred, average='macro') * 100.
      #confidence = np.mean(y_conf) * 100.

      # results_dict = {
      #     'accuracy': accuracy_score(y_true, y_pred) * 100.,
      #     'macro-F1': f1_score(y_true, y_pred, average='macro') * 100.,
      #     'confidence': np.mean(y_conf) * 100.,
      # }
      # print(results_dict)
      # print(classification_report(y_true,y_pred))
      # print(confusion_matrix(y_true,y_pred))
      print("writing the results...")
      class_map={'algeria': 0, 'bahrain': 1, 'egypt': 2, 'iraq': 3, 'jordan': 4, 'ksa': 5, 'kuwait': 6, 'lebanon': 7, 'libya': 8, 'morocco': 9, 'oman': 10, 'palestine': 11, 'qatar': 12, 'sudan': 13, 'syria': 14, 'tunisia': 15, 'uae': 16, 'yemen': 17}
      c={}
      for i in class_map:
        c[class_map[i]]=i
      p=[]
      #a=[]
      for i,k in enumerate(y_pred):
          p.append(c[k])
          #a.append(c[y_true[i]])
      path=input_dir+"/official_results/testa_1.txt"
      #gold_path=input_dir+"/results/AraBERTgold.txt"
      with open(path,"w") as f:
        for r in p:
          f.write(str(r))
          f.write("\n")
      # with open(gold_path,"w") as f:
      #   for r in a:
      #     f.write(str(r))
      #     f.write("\n")
      return output_dicts


    

In [ ]:
path=input_dir
m = Model(path)
XY=0
sentences_train,labels_train=m.extract_data('/NADI2022_Subtask1_TRAIN.tsv',XY=0)

In [ ]:
#sentences_dev,labels_dev=m.extract_data('/NADI2022_Subtask1_DEV.tsv',XY=0)

In [ ]:
m.process_inputs(sentences_train,labels_train)



{'algeria': 0, 'bahrain': 1, 'egypt': 2, 'iraq': 3, 'jordan': 4, 'ksa': 5, 'kuwait': 6, 'lebanon': 7, 'libya': 8, 'morocco': 9, 'oman': 10, 'palestine': 11, 'qatar': 12, 'sudan': 13, 'syria': 14, 'tunisia': 15, 'uae': 16, 'yemen': 17}


If you need to train the model, set train=1 else train=0

In [ ]:
out=m.train_save_load(train=1,retrain=0,label_smoothing = -1,XY=0)#considering dev as test set

- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
#act_out_dev=m.simple_test()

In [ ]:
sentences_test=m.extract_data_test('/NADI2022_Subtask1_TEST-A_Unlabeled.tsv',XY=0)
m.process_inputs_test(sentences_test)
act_out_test=m.simple_test()

Please remember to change the result paths accordingly before running these lines of codes.

In [ ]:
sentences_test=m.extract_data_test('/NADI2022_Subtask1_TEST-B_Unlabeled.tsv',XY=0)
m.process_inputs_test(sentences_test)
act_out_test=m.simple_test()

writing the results...


In [ ]:
#! python NADI2022-ST1-Scorer.py results/AraBERTgold.txt results/AraBERTTestA.txt -verbose